In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
import datetime
from datetime import date
import matplotlib
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from sklearn.preprocessing import StandardScaler, normalize
from sklearn import metrics
from sklearn.mixture import GaussianMixture
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
import warnings
warnings.filterwarnings('ignore')


In [ ]:
data=pd.read_csv('/kaggle/input/customer-personality-analysis/marketing_campaign.csv',sep="\t")

In [ ]:
data

In [ ]:
!pip install dataprep


In [ ]:
from dataprep.eda import plot, plot_correlation, create_report, plot_missing
plot(data)

In [ ]:
data.info()

In [ ]:
data.describe()

**we have missing values in the Income table and as this is a clustring not a predicting problem , we can't really guess what the income is , so we will drop the missing values**

In [ ]:
data = data.dropna()
print("The total number of data-points after removing the rows with missing values are:", len(data))

**Now regarding the Dt_customer , we don't know what date should we use in order to extract number of days for each customer 
so i will just use the most recent record in our data**

In [ ]:
data.columns

In [ ]:
data['Dt_Customer'] = pd.to_datetime(data['Dt_Customer'])


In [ ]:
max_date = data['Dt_Customer'].max()
data['_num_days'] = (max_date - data['Dt_Customer']).dt.days

**Based on the recent dates , I'm going to assume that we live in 2015 .. 
also we notice a lot of values in the categorical features 
here comes the feature engineering part**

In [ ]:
#Feature Engineering
#Age of customer today 
data["Age"] = 2015-data["Year_Birth"]

#Total spendings on various items
data["Spent"] = data["MntWines"]+ data["MntFruits"]+ data["MntMeatProducts"]+ data["MntFishProducts"]+ data["MntSweetProducts"]+ data["MntGoldProds"]

#Deriving living situation by marital status"Alone"
data["Living_With"]=data["Marital_Status"].replace({"Married":"Partner", "Together":"Partner", "Absurd":"Alone", "Widow":"Alone", "YOLO":"Alone", "Divorced":"Alone", "Single":"Alone",})

#Feature indicating total children living in the household
data["Children"]=data["Kidhome"]+data["Teenhome"]

#Feature for total members in the householde
data["Family_Size"] = data["Living_With"].replace({"Alone": 1, "Partner":2})+ data["Children"]

#Feature pertaining parenthood
data["Is_Parent"] = np.where(data.Children> 0, 1, 0)

#Segmenting education levels in three groups
data["Education"]=data["Education"].replace({"Basic":"Undergraduate","2n Cycle":"Undergraduate", "Graduation":"Graduate", "Master":"Postgraduate", "PhD":"Postgraduate"})

#For clarity
data=data.rename(columns={"MntWines": "Wines","MntFruits":"Fruits","MntMeatProducts":"Meat","MntFishProducts":"Fish","MntSweetProducts":"Sweets","MntGoldProds":"Gold"})

#Dropping some of the redundant features
to_drop = ["Marital_Status", "Dt_Customer", "Z_CostContact", "Z_Revenue", "Year_Birth", "ID"]
data = data.drop(to_drop, axis=1)

In [ ]:
data.describe()

In [ ]:
data.columns

In [ ]:
from matplotlib import colors
from matplotlib.colors import ListedColormap
sns.set(rc={"axes.facecolor":"#FFF9ED","figure.facecolor":"#FFF9ED"})
pallet = ["#682F2F", "#9E726F", "#D6B2B1", "#B9C0C9", "#9F8A78", "#F3AB60"]
cmap = colors.ListedColormap(["#682F2F", "#9E726F", "#D6B2B1", "#B9C0C9", "#9F8A78", "#F3AB60"])
#Plotting following features
To_Plot = [ "Income", "Recency", "_num_days", "Age", "Spent", "Is_Parent"]
print("Reletive Plot Of Some Selected Features: A Data Subset")
plt.figure()
sns.pairplot(data[To_Plot], hue= "Is_Parent",palette= (["#682F2F","#F3AB60"]))
#Taking hue 
plt.show()

**conclusion : We have a lot of work to do with outliers**

In [ ]:
q1_income = np.percentile(data['Income'], 25)
q3_income = np.percentile(data['Income'], 75)

q1_age = np.percentile(data['Age'], 25)
q3_age = np.percentile(data['Age'], 75)

iqr_income = q3_income - q1_income
iqr_age = q3_age - q1_age

threshold = 1.5

outliers_income = data[(data['Income'] < q1_income - threshold * iqr_income) | (data['Income'] > q3_income + threshold * iqr_income)]
outliers_age = data[(data['Age'] < q1_age - threshold * iqr_age) | (data['Age'] > q3_age + threshold * iqr_age)]

print("Outliers in 'Income':")
print(outliers_income)

print("\nOutliers in 'Age':")
print(outliers_age)


In [ ]:
# Drop the maximum value in 'Income'
max_income = data['Income'].max()
data = data[data['Income'] != max_income]

# Remove outliers in 'Age'
q1_age = np.percentile(data['Age'], 25)
q3_age = np.percentile(data['Age'], 75)
iqr_age = q3_age - q1_age
threshold = 1.5

lower_bound = q1_age - threshold * iqr_age
upper_bound = q3_age + threshold * iqr_age

data = data[(data['Age'] >= lower_bound) & (data['Age'] <= upper_bound)]


1. Stars: Old customers with high income and high spending nature.
2. Neet Attention: New customers with below-average income and low spending nature. 
3. High Potential: New customers with high income and high spending nature.
4. Leaky Bucket: Old customers with below-average income and a low spending nature.


In [ ]:
scaler=StandardScaler()
dataset_temp=data[['Income','_num_days','Spent']]
X_std=scaler.fit_transform(dataset_temp)
X = normalize(X_std,norm='l2')

gmm=GaussianMixture(n_components=4, covariance_type='spherical',max_iter=2000, random_state=5).fit(X)
labels = gmm.predict(X)
dataset_temp['Cluster'] = labels
dataset_temp=dataset_temp.replace({0:'Stars',1:'Need attention',2:'High potential',3:'Leaky bucket'})
data = data.merge(dataset_temp.Cluster, left_index=True, right_index=True)

pd.options.display.float_format = "{:.0f}".format
summary=data[['Income','Spent','_num_days','Cluster']]
summary.set_index("Cluster", inplace = True)
summary=summary.groupby('Cluster').describe().transpose()
summary.head()

In [ ]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

# Select the features for clustering
features = ['Income', '_num_days', 'Spent']

# Create a new dataframe with the selected features
data_cluster = data[features].copy()

# Scale the data using StandardScaler
scaler = StandardScaler()
data_scaled = scaler.fit_transform(data_cluster)

# Apply k-means clustering
k = 4  # Number of clusters
kmeans = KMeans(n_clusters=k, random_state=5)
labels = kmeans.fit_predict(data_scaled)

# Map cluster labels to names
cluster_names = {0: 'Stars', 1: 'Need attention', 2: 'High potential', 3: 'Leaky bucket'}
cluster_labels = [cluster_names[label] for label in labels]

# Assign cluster labels to the original dataset
data['Cluster'] = cluster_labels

# Create a summary dataframe
summary = data.groupby('Cluster')[features].describe().transpose()

summary


**Well , K means makes more sense to me !**

In [ ]:
PLOT = go.Figure()
for C in list(data.Cluster.unique()):
    

    PLOT.add_trace(go.Scatter3d(x = data[data.Cluster == C]['Income'],
                                y = data[data.Cluster == C]['_num_days'],
                                z = data[data.Cluster == C]['Spent'],                        
                                mode = 'markers',marker_size = 6, marker_line_width = 1,
                                name = str(C)))
PLOT.update_traces(hovertemplate='Income: %{x} <br>Seniority: %{y} <br>Spending: %{z}')

    
PLOT.update_layout(width = 800, height = 800, autosize = True, showlegend = True,
                   scene = dict(xaxis=dict(title = 'Income', titlefont_color = 'black'),
                                yaxis=dict(title = '_num_days', titlefont_color = 'black'),
                                zaxis=dict(title = 'Spent', titlefont_color = 'black')),
                   font = dict(family = "Gilroy", color  = 'black', size = 12))

In [ ]:
pal = ["#682F2F","#B9C0C9", "#9F8A78","#F3AB60"]
pl = sns.countplot(x=data["Cluster"], palette= pal)
pl.set_title("Distribution Of The Clusters")
plt.show()

In [ ]:
cut_labels_Age = ['Young', 'Adult', 'Mature', 'Senior']
cut_bins = [0, 30, 45, 65, 120]
data['Age_group'] = pd.cut(data['Age'], bins=cut_bins, labels=cut_labels_Age)
#Create Income segment
cut_labels_Income = ['Low income', 'Low to medium income', 'Medium to high income', 'High income']
data['Income_group'] = pd.qcut(data['Income'], q=4, labels=cut_labels_Income)
#Create Seniority segment
cut_labels_Seniority = ['New customers', 'Discovering customers', 'Experienced customers', 'Old customers']
data['Seniority_group'] = pd.qcut(data['_num_days'], q=4, labels=cut_labels_Seniority)
data=data.drop(columns=['Age','Income','_num_days'])

In [ ]:
cut_labels = ['Low consumer', 'Frequent consumer', 'Biggest consumer']
data['Wines_segment'] = pd.qcut(data['Wines'][data['Wines']>0],q=[0, .25, .75, 1], labels=cut_labels).astype("object")
data['Fruits_segment'] = pd.qcut(data['Fruits'][data['Fruits']>0],q=[0, .25, .75, 1], labels=cut_labels).astype("object")
data['Meat_segment'] = pd.qcut(data['Meat'][data['Meat']>0],q=[0, .25, .75, 1], labels=cut_labels).astype("object")
data['Fish_segment'] = pd.qcut(data['Fish'][data['Fish']>0],q=[0, .25, .75, 1], labels=cut_labels).astype("object")
data['Sweets_segment'] = pd.qcut(data['Sweets'][data['Sweets']>0],q=[0, .25, .75, 1], labels=cut_labels).astype("object")
data['Gold_segment'] = pd.qcut(data['Gold'][data['Gold']>0],q=[0, .25, .75, 1], labels=cut_labels).astype("object")
data.replace(np.nan, "Non consumer",inplace=True)
data.drop(columns=['Spent','Wines','Fruits','Meat','Fish','Sweets','Gold'],inplace=True)
data = data.astype(object)

In [ ]:
data.columns

In [ ]:
import pandas as pd
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', 999)
pd.options.display.float_format = "{:.3f}".format

association = data.copy()
df = pd.get_dummies(association)
min_support = 0.08
max_len = 10
frequent_items = apriori(df, use_colnames=True, min_support=min_support, max_len=max_len + 1)
rules = association_rules(frequent_items, metric='lift', min_threshold=1)

product = 'Wines'  
segment = 'Biggest consumer'  
target = f"{{'{product}_segment_{segment}'}}"
results = rules[rules['consequents'].astype(str).str.contains(target, na=False)].sort_values(by='confidence', ascending=False)
results.head()
